# Premiers tests dans l'implémentation de la méthode ROME.

### Résumé de ce que j'ai compris de l'article

La Méthode ROME fonctionne en gros sur un système key-value qu'on retrouve dans les poids du modèle. On assaie donc d'insérer un couple (k*,v*) à un endroit précis dans notre réseau de neurone.
En pratique:
1. On séléctionne la clé k en faisant une moyenne sur des textes tirés de Wikipédia, j'avoue que je saisis pas trop comment encore.
2. On résoud un problème d'optimisation afin de trouver v associé, voir l'équation sur l'article
3. On insère v* ainsi obtenu.
-> L'insertion se fait visiblement à la couche 18, puisque c'est là qu'on remarque un changement du comportement des couches de MLP qui switchent et commence à agir comme des valeurs (v) et non plus comme des clés (k)

Ils commencent par calculer les moments d'ordre 2 (voir les maths dans l'article) en utilisant 10000 échantillons de couches cachées (k), que l'on compute en utilisant des tokesnde toutes les pages Wikipedia (2020-05-01 snapshot)-> On ne se limite pas aux sujets mais tous les tokens sont aussi inclus visiblement.
-> Le text complet de chaque article est passé dans le transformer jusq'à ce qu'on atteigne la limite, puis on stocke les activations de la partie MLP (k) pour chaque token de l'article en question.
-> Et on repète le process jusqu'à obtenir 10 000 vecteur k. Ensuite on calcule les moments d'ordre 2 E[kk^T]

Bon c'est pas hyper clair mdr


## Etape 1: On essaie de compute les moments d'ordre 2 à partir de Wikipedia

In [2]:
from datasets import load_dataset

ds_name = 'wikitext'

raw_ds = load_dataset(ds_name, dict(wikitext="wikitext-103-raw-v1", wikipedia="20200501.en")[ds_name])
raw_ds

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 1801350
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [5]:
raw_ds["test"][:30]

{'text': ['',
  ' = Robert Boulter = \n',
  '',
  ' Robert Boulter is an English film , television and theatre actor . He had a guest @-@ starring role on the television series The Bill in 2000 . This was followed by a starring role in the play Herons written by Simon Stephens , which was performed in 2001 at the Royal Court Theatre . He had a guest role in the television series Judge John Deed in 2002 . In 2004 Boulter landed a role as " Craig " in the episode " Teddy \'s Story " of the television series The Long Firm ; he starred alongside actors Mark Strong and Derek Jacobi . He was cast in the 2005 theatre productions of the Philip Ridley play Mercury Fur , which was performed at the Drum Theatre in Plymouth and the Menier Chocolate Factory in London . He was directed by John Tiffany and starred alongside Ben Whishaw , Shane Zaza , Harry Kent , Fraser Ayres , Sophie Stanton and Dominic Hall . \n',
  ' In 2006 , Boulter starred alongside Whishaw in the play Citizenship written by Ma

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from tok_dataset import (
    TokenizedDataset,
    dict_to_,
    flatten_masked_batch,
    length_collation,
)

tokenizer = AutoTokenizer.from_pretrained('gpt2')

ds = TokenizedDataset(raw_ds["train"], tokenizer, maxlen=1000)

{'input_ids': tensor([]),
 'position_ids': tensor([]),
 'attention_mask': tensor([])}